Opis:...

Generisanje datafrejma - funkcije za pripremu

In [1]:
import numpy as np
import pandas as pd
import json
import os
import itertools
from tqdm import tqdm

In [2]:
def load_annotations(annotations_folder_path):
    annotations = []
    for annotation_name in tqdm(os.listdir(annotations_folder_path)):
        annotation_path = os.path.join(annotations_folder_path, annotation_name)
        annotation = json.load(open(annotation_path))
        annotation["filename"] = annotation_name.replace(".json", "")
        annotations.append(annotation)
    return annotations

In [3]:
def get_flattened_dict(input_dict, base_key="", output_dict={}, index_value=None):
    
    for key in input_dict.keys():
        
        if base_key == "":
            full_key = key     
        else:
            if index_value is None:
                full_key = base_key + "/" + key
            else:
                full_key = base_key + "/" + key + "/" + index_value

        if isinstance(input_dict[key], dict):
            get_flattened_dict(input_dict[key], full_key, output_dict)
            
        elif isinstance(input_dict[key], list):
            for index, item in enumerate(input_dict[key]):
                full_list_key = full_key
                get_flattened_dict(item, full_list_key, output_dict, str(index))
                    
        else:
            output_dict[full_key] = input_dict[key]

In [4]:
KEY_ORDER_DICT = {
    "description": "00-00000-00",
    "tags":[],
    "size": {
        "width": "02-0000-00",
        "height": "03-0000-00"
    },
    "objects": []
}

In [5]:
def get_column_index(split_key, key_order_dict, level=0, id_value=None):
    key = split_key[level]
   
        
    if isinstance(key_order_dict[key], dict):
        return get_column_index(split_key, key_order_dict[key], level+1, id_value)
    
    else:
        column_index = key_order_dict[key]
        if id_value is not None:
            split_column_index = column_index.split("-")
            split_column_index[1] = split_column_index[1].replace("id", id_value)[-5:]
            column_index = "-".join(split_column_index)
        return column_index

In [6]:


def get_sorted_columns(annotation_df_columns):
    sorted_annotation_df_cols = []
    for annotation_df_col in annotation_df_columns:
        split_col = annotation_df_col.split("/")
        col_index = get_column_index(split_col, KEY_ORDER_DICT, 0)
        sorted_annotation_df_cols.append(str(col_index) + "#" + annotation_df_col)
    sorted_annotation_df_cols.sort()
    sorted_annotation_df_cols = [sorted_annotation_df_col.split("#")[1] for sorted_annotation_df_col in sorted_annotation_df_cols]
    return sorted_annotation_df_cols



In [7]:
def get_annotation_df(annotations):
    annotation_dicts = []
    for annotation in annotations:
        annotation_dict = {}
        get_flattened_dict(annotation, "", annotation_dict)
        annotation_dicts.append(annotation_dict)
    annotation_dicts_keys = set(list(itertools.chain.from_iterable([list(annotation_dict.keys()) for annotation_dict in annotation_dicts])))
    annotation_dicts_keys = get_sorted_columns(list(annotation_dicts_keys))
    annotation_df_dict = {key: [] for key in annotation_dicts_keys}
    for annotation_dict in annotation_dicts:
        for key in annotation_df_dict:
            value = annotation_dict[key] if key in annotation_dict else np.nan
            annotation_df_dict[key].append(value)
    json_df = pd.DataFrame(annotation_df_dict)
    return json_df

Generisanje datafrejma - sample dataset

In [21]:
train_annotations_path = "bdd100ksample/val/ann"
train_annotations = load_annotations(train_annotations_path)



df2 = pd.DataFrame.from_dict(train_annotations, orient='columns')


df2

100%|███████████████████████████████████████████████████████████████████████████████| 109/109 [00:00<00:00, 432.55it/s]


,description,tags,size,objects,filename
0,,"[{'id': 10789675, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 88945149, 'classId': 6508807, 'descrip...",b1f6c103-8b75ea3e.jpg
1,,"[{'id': 10790092, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 88948965, 'classId': 6508804, 'descrip...",b25fb716-78d8d49b.jpg
2,,"[{'id': 10790221, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 88950127, 'classId': 6508807, 'descrip...",b26f9b1e-3e172ced.jpg
3,,"[{'id': 10790369, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 88952159, 'classId': 6508807, 'descrip...",b289f2e8-cec8700d.jpg
4,,"[{'id': 10791039, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 88959006, 'classId': 6508807, 'descrip...",b313a4dc-217f1233.jpg
...,...,...,...,...,...
104,,"[{'id': 10818874, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 89256297, 'classId': 6508804, 'descrip...",c9b9fef4-a58bf3d5.jpg
105,,"[{'id': 10818931, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 89256888, 'classId': 6508800, 'descrip...",c9c31e57-45b42452.jpg
106,,"[{'id': 10819441, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 89261734, 'classId': 6508807, 'descrip...",ca28f275-03f8cef5.jpg
107,,"[{'id': 10819429, 'tagId': 26739, 'name': 'wea...","{'height': 720, 'width': 1280}","[{'id': 89261594, 'classId': 6508807, 'descrip...",ca28f275-cac4f254.jpg


Pregled objekata sample dataseta

Train

In [23]:
objetcs_id_dict = {
    0: 'car',
    1: 'bus',
    2: 'drivable area',
    3: 'lane',
    4: 'traffic sign',
    5: 'truck',
    6: 'person',
    7: 'traffic light',
    8: 'rider',
    9: 'bike',
    10:'motor',
    11:'train'
}

In [24]:
df_totals = pd.DataFrame({
    'car':[0],
    'bus':[0],
    'drivable area':[0],
    'lane':[0],
    'traffic sign':[0],
    'truck':[0],
    'person':[0],
    'traffic light':[0],
    'rider':[0],
    'bike':[0],
    'motor':[0],
    'train':[0]
})
df_per_picture = pd.DataFrame({
    'car':[0],
    'bus':[0],
    'drivable area':[0],
    'lane':[0],
    'traffic sign':[0],
    'truck':[0],
    'person':[0],
    'traffic light':[0],
    'rider':[0],
    'bike':[0],
    'motor':[0],
    'train':[0]
})

for i in range(len(df2.index)): 
    picture_flags = [False, False, False, False, False, False, False, False, False, False, False, False]
    for o in df2.loc[i,'objects']:
        o_id = (o['classId']-6508800)
        df_totals.loc[0,objetcs_id_dict[o_id]] = df_totals.loc[0,objetcs_id_dict[o_id]] + 1
        if not picture_flags[o_id]:
            df_per_picture.loc[0,objetcs_id_dict[o_id]] = df_per_picture.loc[0,objetcs_id_dict[o_id]] + 1
            picture_flags[o_id] = True
        

In [25]:
df_totals

,car,bus,drivable area,lane,traffic sign,truck,person,traffic light,rider,bike,motor,train
0,1257,12,202,885,426,55,169,336,8,13,15,0


In [26]:
df_per_picture

,car,bus,drivable area,lane,traffic sign,truck,person,traffic light,rider,bike,motor,train
0,109,11,107,105,97,37,44,74,7,10,4,0


Val

Transformacija anotacija u yolo format

Train

In [20]:
for i in range(len(df2.index)):
    save_location = "bdd100ksample/train/labels/" + df2.loc[i,'filename'].rstrip('.jpg') + ".txt"
    file = open(save_location,'w')
    for o in df2.loc[i,'objects']:
        if o['classId'] - 6508800 != 7:
            continue
        str_line = ""
        str_line += ("0 ")
        str_line += (str((o['points']['exterior'][1][0]+o['points']['exterior'][0][0])/(2*1280)) + " ")
        str_line += (str((o['points']['exterior'][1][1]+o['points']['exterior'][0][1])/(2*720)) + " ")
        str_line += (str((o['points']['exterior'][1][0]-o['points']['exterior'][0][0])/(1280)) + " ")
        str_line += (str((o['points']['exterior'][1][1]-o['points']['exterior'][0][1])/(720)) + "\n")
        file.write(str_line)
    file.close()

Val

Treniranje modela nad sample skupom

In [2]:
#!pip install ultralytics
from ultralytics import YOLO

from IPython.display import display, Image

In [4]:
model = YOLO('yolov8n.yaml')

In [5]:
results = model.train(data='bdd100ksample/data.yaml', batch = 2, imgsz=1280)

Ultralytics YOLOv8.1.24 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=bdd100ksample/data.yaml, epochs=100, time=None, patience=100, batch=2, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

train: Scanning C:\Users\these\Desktop\tem\bdd100ksample\train\labels.cache... 927 images, 27 backgrounds, 0 corrupt: 1
val: Scanning C:\Users\these\Desktop\tem\bdd100ksample\val\labels.cache... 109 images, 35 backgrounds, 0 corrupt: 100%|


Plotting labels to runs\detect\train22\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1280 train, 1280 val
Using 0 dataloader workers
Logging results to runs\detect\train22
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      3.314      37.68      2.457         32       1280:   0%|          | 2/464 [00:26<1:41:06, 1


KeyboardInterrupt: 

Prikaz rezultata nad test skupom

In [3]:
import cv2

In [ ]:
imgs = []
model = YOLO('mod2.pt')
for img in tqdm(os.listdir( 'bdd100ksample/test/images')):
    img_path = os.path.join('bdd100ksample/test/images', img)
    imgs.append(img_path)
    
for img in imgs:
    imgr = cv2.imread(img,1)
    results = model(img)[0]
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        cv2.rectangle(imgr, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
        cv2.putText(imgr, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow('',imgr)
        if cv2.waitKey(0) == ord('q'):
            break
    if cv2.waitKey(0) == ord('q'):
        cv2.destroyAllWindows()
        break      
    cv2.destroyAllWindows()

Generisanje datafrejma - ceo dataset

Train

In [8]:
train_annotations_path = "bdd100k/train/ann"
train_annotations = load_annotations(train_annotations_path)

df = pd.DataFrame.from_dict(train_annotations, orient='columns')

df

100%|███████████████████████████████████████████████████████████████████████████| 69736/69736 [02:17<00:00, 506.18it/s]


In [ ]:
df_totals2 = pd.DataFrame({
    'car':[0],
    'bus':[0],
    'drivable area':[0],
    'lane':[0],
    'traffic sign':[0],
    'truck':[0],
    'person':[0],
    'traffic light':[0],
    'rider':[0],
    'bike':[0],
    'motor':[0],
    'train':[0]
})
df_per_picture2 = pd.DataFrame({
    'car':[0],
    'bus':[0],
    'drivable area':[0],
    'lane':[0],
    'traffic sign':[0],
    'truck':[0],
    'person':[0],
    'traffic light':[0],
    'rider':[0],
    'bike':[0],
    'motor':[0],
    'train':[0]
})

for i in range(len(df.index)): 
    picture_flags = [False, False, False, False, False, False, False, False, False, False, False, False]
    for o in df.loc[i,'objects']:
        o_id = (o['classId']-6508800)
        df_totals2.loc[0,objetcs_id_dict[o_id]] = df_totals2.loc[0,objetcs_id_dict[o_id]] + 1
        if not picture_flags[o_id]:
            df_per_picture2.loc[0,objetcs_id_dict[o_id]] = df_per_picture2.loc[0,objetcs_id_dict[o_id]] + 1
            picture_flags[o_id] = True

In [ ]:
df_totals2

In [ ]:
df_per_picture2

Val

Transformacija sample dataseta

1. Brisanje svih slika i anotacija koje ne sadrze semafor u sebi

In [12]:
jpg_dict = {}

for picture in tqdm(os.listdir('bdd100ksample/train/images')):
       jpg_dict[picture] = False


100%|████████████████████████████████████████████████████████████████████████| 69736/69736 [00:00<00:00, 757942.97it/s]

69736


In [14]:
for ind in df2.index:
    for obj in df2['objects'][ind]:
        if (obj['classId']-6508800)==7:
            jpg_dict[df2['filename'][ind]] = True
            break


In [19]:
for pair in jpg_dict:
    if not jpg_dict[pair]:
        os.remove('bdd100k/train/ann/'+pair+'.json')


2. Kopiranje slika iz celog dataseta u sample

In [13]:
jpg_dict = {}

for ind in df.index:
    jpg_dict[df['filename'][ind]] = False
    

69736


In [ ]:
for picture in tqdm(os.listdir('bdd100ksample/train/images')):
       jpg_dict[picture] = True

In [18]:
import shutil
sem_picture_num = 900-435
no_sem_picture_num = 27
for ind in df.index:
    pic_filename = df['filename'][ind]
    
    if jpg_dict[pic_filename]:
        continue
        
    is_sem_pic = False
    for obj in df['objects'][ind]:
        if (obj['classId']-6508800)==7:
            shutil.copyfile('bdd100k/train/ann/'+pic_filename+'.json', 'bdd100ksample/train/ann/'+pic_filename+'.json')
            shutil.copyfile('bdd100k/train/images/'+pic_filename, 'bdd100ksample/train/images/'+pic_filename)
            sem_picture_num-=1
            is_sem_pic = True
            break
    if not is_sem_pic and no_sem_picture_num > 0:
        shutil.copyfile('bdd100k/train/ann/'+pic_filename+'.json', 'bdd100ksample/train/ann/'+pic_filename+'.json')
        shutil.copyfile('bdd100k/train/images/'+pic_filename, 'bdd100ksample/train/images/'+pic_filename)
        no_sem_picture_num-=1
    if no_sem_picture_num==0:
        break

Prikaz rezultata sa poboljsanim sample datasetom ...

Transformacija celog dataseta

1. Smanjivanje broja slika bez semafora na 1000

In [ ]:
total_no_sem = 1000
for ind in df.index:
    to_delete = True
    for obj in df['objects'][ind]:
        if (obj['classId']-6508800)==7:
            to_delete = False
            break
    if total_no_sem>0:
        total_no_sem -=1
        to_delete = False
    if to_delete:
        os.remove('bdd100k/train/ann/'+pair+'.json')
        os.remove('bdd100k/train/images/'+pair)


ponovno ucitavanje dataseta

In [ ]:
train_annotations_path = "bdd100k/train/ann"
train_annotations = load_annotations(train_annotations_path)

df = pd.DataFrame.from_dict(train_annotations, orient='columns')

df

2. Transformacija u yolo format

In [39]:
#srediti jos

for i in range(len(df.index)):
    save_location = "bdd100k/train/labels/" + df.loc[i,'filename'].rstrip('.jpg') + ".txt"
    file = open(save_location,'w')
    for o in df.loc[i,'objects']:
        if o['classId'] - 6508800 != 7:
            continue
        str_line = ""
        str_line += ("0 ")
        str_line += (str((o['points']['exterior'][1][0]+o['points']['exterior'][0][0])/(2*1280)) + " ")
        str_line += (str((o['points']['exterior'][1][1]+o['points']['exterior'][0][1])/(2*720)) + " ")
        str_line += (str((o['points']['exterior'][1][0]-o['points']['exterior'][0][0])/(1280)) + " ")
        str_line += (str((o['points']['exterior'][1][1]-o['points']['exterior'][0][1])/(720)) + "\n")
        file.write(str_line)
    file.close()


Prikaz rezultata nad celim skupom...

In [34]:
for i in range(len(df.index)): 
    img = cv2.imread("bdd100k/train/images/"+df.loc[i,'filename'],1)
   # print(i)
    for o in df.loc[i,'objects']:
        o_id = (o['classId']-6508800)
        if o_id != 7:
            continue
        cv2.rectangle(img,(o['points']['exterior'][0][0],o['points']['exterior'][0][1]),(o['points']['exterior'][1][0],o['points']['exterior'][1][1]),(0,0,255),2)
        cv2.imshow(df.loc[i,'filename'],img)
        if cv2.waitKey(0) == ord('q'):
            break
    if cv2.waitKey(0) == ord('q'):
        cv2.destroyAllWindows()
        break      
    cv2.destroyAllWindows()
        




In [2]:
import os


import cv2


VIDEOS_DIR = os.path.join('.', 'videos')

video_path = 'vid5.mp4'
video_path_out = '{}_out.mp4'.format(video_path)

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))



# Load a model
model = YOLO('mod.pt')  # load a custom model

threshold = 0.5

while ret:

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()


0: 736x1280 (no detections), 7847.1ms
Speed: 20.3ms preprocess, 7847.1ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 7451.6ms
Speed: 14.4ms preprocess, 7451.6ms inference, 0.0ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 7928.8ms
Speed: 20.2ms preprocess, 7928.8ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 7907.9ms
Speed: 10.2ms preprocess, 7907.9ms inference, 7.6ms postprocess per image at shape (1, 3, 736, 1280)



KeyboardInterrupt: 